# Analyze data using the Orbit API

This notebook shows how to analyze images using the Orbit API. A model has been generated and uploaded to the server.

### Description

The following section shows:
  * how to connect to OMERO from Orbit.
  * how to retrieve an Orbit model already saved in OMERO.
  * how to segment an Image.
  * how to save the generated Regions of Interest (ROI).

### Setup: Add Orbit to the Classpath

In [25]:
%classpath add jar /srv/conda/orbit/lib/*

## Enter credentials. Do not re-run this cell. 

In [26]:
g = new EasyForm("Enter credentials and continue to the next cell. Do not re-run this cell")
g.addTextField("Server").onInit({g['Server'] = "wss://outreach.openmicroscopy.org/omero-ws"})
g.addTextField("UserName")
g.addPasswordField("Password")
g

## Collect the credentials and Connect to OMERO

In [27]:
import com.actelion.research.orbit.imageprovider.ImageProviderOmero
import com.actelion.research.orbit.imageprovider.OmeroConf
import com.actelion.research.orbit.imageAnalysis.dal.DALConfig

String hostName = g['Server']
String userName = g['UserName']
String password = g['Password']

OmeroConf conf = new OmeroConf(hostName, 443, true)
conf.setUseWebSockets(true)
imageProvider = new ImageProviderOmero(conf)

imageProvider.authenticateUser(userName, password)

//Set the provider
DALConfig.setImageProvider(imageProvider)

println("Connected ")

Connected 


null

## Select the Image. Do not re-run this cell.

In [4]:
ga = new EasyForm("Select the Image to analyze")
ga.addTextField("ImageID")
ga

In [28]:
omeroImageId = ga['ImageID'].toLong()

10001

## Load the Image from the OMERO server

Load the image so we can get information like size.

In [29]:
import omero.gateway.Gateway
import omero.gateway.SecurityContext
import omero.gateway.facility.BrowseFacility

Gateway gateway = imageProvider.getGatewayAndCtx().getGateway()
SecurityContext ctx = imageProvider.getGatewayAndCtx().getCtx()

browse = gateway.getFacility(BrowseFacility)

image = browse.getImage(ctx, omeroImageId)

omero.gateway.model.ImageData (id=10001)

## Load the Orbit model

We use the most recent model added. Only the models owned by the currently logged in user are loaded.

In [31]:
import java.util.List
import com.actelion.research.orbit.beans.RawAnnotation
import com.actelion.research.orbit.imageAnalysis.models.OrbitModel

// Load Models that I own. OMERO annotations of type: Model
imageProvider.setOnlyOwnerObjects(true)
List<RawAnnotation> annotations = imageProvider.LoadRawAnnotationsByType(RawAnnotation.ANNOTATION_TYPE_MODEL)
println("Found " + annotations.size() + " files")
//Use the latest uploaded model
int fileAnnId = 0
for (RawAnnotation ra : annotations) {
    id = ra.getRawAnnotationId()
    if (id > fileAnnId) {
        fileAnnId = id
    }
}
model = OrbitModel.LoadFromOrbit(fileAnnId)
println("Loaded Model: " + model.getName())

Found 2 files
Loaded Model: orbit3.63-img_10001 [svs]


null

## Segment the selected image.

We analyze a 500x500 square around the center of the image.

In [36]:
import com.actelion.research.orbit.imageAnalysis.models.RectangleExt

pixels = image.getDefaultPixels()
w = 250
h = 250
cx = (int) (pixels.getSizeX()/2)
cy = (int) (pixels.getSizeY()/2)
region = new RectangleExt(cx-w, cy-w, 2*w, 2*h)

com.actelion.research.orbit.imageAnalysis.models.RectangleExt[x=48249,y=22583,width=500,height=500]

In [37]:
import com.actelion.research.orbit.imageAnalysis.models.SegmentationResult
import com.actelion.research.orbit.imageAnalysis.utils.OrbitHelper
import com.actelion.research.orbit.beans.RawDataFile
import com.actelion.research.orbit.imageAnalysis.components.RecognitionFrame


// Perform the segmentation
RawDataFile rdf = imageProvider.LoadRawDataFile((int) omeroImageId)
RecognitionFrame rf = new RecognitionFrame(rdf, false)
result = OrbitHelper.Segmentation(rf, (int) omeroImageId, model, null, 1, true, region)

ending: svs [series 1]  format: SVS
type: 
ending: svs [series 1]  format: SVS
type: 
TaskResult:
Number of objects: 228


com.actelion.research.orbit.imageAnalysis.models.SegmentationResult@46d448a4

## Save the ROIs back to OMERO

We retrieve the security context from the ``imageProvider``.

In [38]:
import java.util.ArrayList
import java.util.List
import omero.gateway.Gateway
import omero.gateway.SecurityContext

import java.awt.Shape

import omero.model.ImageI
import omero.model.PolygonI
import omero.model.RoiI

import static omero.rtypes.rstring
import static omero.rtypes.rint
import omero.gateway.facility.BrowseFacility
import omero.gateway.facility.DataManagerFacility

// Retrieve the security context
Gateway gateway = imageProvider.getGatewayAndCtx().getGateway()
SecurityContext ctx = imageProvider.getGatewayAndCtx().getCtx()

// Save the shapes
List<RoiI> roisToSave = new ArrayList<RoiI>()
for (Shape shape: result.shapeList) {
    // can cast shape to Polygon or simply listPoints
    String points = shape.listPoints()

    // Create Polygon in OMERO
    p = new PolygonI()
    // Convert "x, y; x, y" format to "x, y, x, y" for OMERO
    points = points.replace(";", ",")
    p.setPoints(rstring(points))
    p.setTheT(rint(0))
    p.setTheZ(rint(0))
    p.setStrokeColor(rint(-65281))   // yellow

    // Add each shape to an ROI on the Image
    ImageI image = new ImageI(omeroImageId, false)
    RoiI roi = new RoiI()
    roi.setImage(image)
    roi.addShape(p)
    roisToSave.add(roi)

}
gateway.getUpdateService(ctx).saveAndReturnArray(roisToSave)

[omero.model.RoiI@1ed9247f, omero.model.RoiI@7a2cce68, omero.model.RoiI@657e4a2b, omero.model.RoiI@213f20b5, omero.model.RoiI@76920231, omero.model.RoiI@6a51dd05, omero.model.RoiI@7d69fd76, omero.model.RoiI@258b50e6, omero.model.RoiI@43ee5dec, omero.model.RoiI@27330961, omero.model.RoiI@1503d2bc, omero.model.RoiI@6a70bc9a, omero.model.RoiI@5bc08d05, omero.model.RoiI@6487ef1d, omero.model.RoiI@6f1a5703, omero.model.RoiI@61b216, omero.model.RoiI@5f16bfba, omero.model.RoiI@4325c42, omero.model.RoiI@39adb188, omero.model.RoiI@345145a9, omero.model.RoiI@5ed0c7f6, omero.model.RoiI@48965022, omero.model.RoiI@1ce39949, omero.model.RoiI@2221fb71, omero.model.RoiI@7a1667b4, omero.model.RoiI@7b2f94, omero.model.RoiI@d2c703, omero.model.RoiI@2d054e1b, omero.model.RoiI@6467a7c8, omero.model.RoiI@3d39235f, omero.model.RoiI@2b451124, omero.model.RoiI@2c37e7c1, omero.model.RoiI@75374c68, omero.model.RoiI@1e0aa4a, omero.model.RoiI@2af5763, omero.model.RoiI@11b5a118, omero.model.RoiI@5eaeeb4a, omero.mod

## Close the connection

In [39]:
imageProvider.close()

null

### License (BSD 2-Clause)
Copyright (C) 2019-2020 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.